# 📊 NEPSE Floor Sheet Analysis Dashboard

## 🎯 Market Manipulation & Broker Activity Detection

This comprehensive analysis tool helps identify suspicious trading patterns and broker activities in the Nepal Stock Exchange (NEPSE) floor sheet data.

### 🔍 Analysis Features:
- **🚨 Broker Dominance Detection**: Identify when single brokers control large volumes
- **🔄 Wash Trading Detection**: Spot same brokers on both buy/sell sides
- **📈 Volume Surge Analysis**: Track unusual volume spikes and responsible brokers
- **💰 Aggressive Buying Detection**: Find brokers consistently buying at higher prices
- **📋 Net Holdings Analysis**: Track broker positions and share turnover

### ⏰ Timeframe Options:
Daily | Weekly | Monthly | Quarterly | Yearly | 15-Day Analysis

---

## 📚 Import Required Libraries

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
import os
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("📊 Ready for NEPSE Floor Sheet Analysis")

## 📂 Data Loading and Preprocessing

In [ ]:
def load_floorsheet_data(data_path='data/'):
    """
    Load all floor sheet CSV files from the data directory
    """
    try:
        # Get all CSV files in the data directory
        csv_files = glob.glob(os.path.join(data_path, "*.csv"))
        
        if not csv_files:
            print("❌ No CSV files found in the data directory!")
            print("Please run the scraper first: python scrape_floorsheet.py")
            return pd.DataFrame()
        
        print(f"📁 Found {len(csv_files)} CSV files")
        
        # Load and combine all CSV files
        dataframes = []
        for file in csv_files:
            try:
                df = pd.read_csv(file)
                # Extract date from filename
                filename = os.path.basename(file)
                if 'nepal_stock_floorsheet_' in filename:
                    date_str = filename.replace('nepal_stock_floorsheet_', '').replace('.csv', '')
                    df['Date'] = date_str
                dataframes.append(df)
                print(f"✅ Loaded: {filename}")
            except Exception as e:
                print(f"❌ Error loading {file}: {str(e)}")
        
        if not dataframes:
            print("❌ No valid data found!")
            return pd.DataFrame()
        
        # Combine all dataframes
        combined_df = pd.concat(dataframes, ignore_index=True)
        
        # Clean and standardize column names
        combined_df.columns = combined_df.columns.str.strip()
        
        return combined_df
        
    except Exception as e:
        print(f"❌ Error in load_floorsheet_data: {str(e)}")
        return pd.DataFrame()

# Load the data
print("🔄 Loading floor sheet data...")
df = load_floorsheet_data()

if not df.empty:
    print(f"✅ Successfully loaded {len(df)} records")
    print(f"📊 Data shape: {df.shape}")
    print(f"📅 Date range: {df['Date'].min()} to {df['Date'].max()}")
    display(df.head())
else:
    print("❌ No data loaded. Please check your data directory.")

In [ ]:
def preprocess_data(df):
    """
    Clean and preprocess the floor sheet data
    """
    if df.empty:
        return df
    
    try:
        # Convert Date column to datetime
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Clean numeric columns
        numeric_columns = ['Quantity', 'Rate (Rs)', 'Amount (Rs)']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Remove rows with missing critical data
        df = df.dropna(subset=['Stock Symbol', 'Buyer', 'Seller'])
        
        # Add time-based columns for analysis
        df['Year'] = df['Date'].dt.year
        df['Quarter'] = df['Date'].dt.quarter
        df['Month'] = df['Date'].dt.month
        df['Week'] = df['Date'].dt.isocalendar().week
        df['DayOfYear'] = df['Date'].dt.dayofyear
        
        # Add period columns for easy filtering
        df['YearQuarter'] = df['Year'].astype(str) + '-Q' + df['Quarter'].astype(str)
        df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)
        df['YearWeek'] = df['Date'].dt.to_period('W').astype(str)
        
        print("✅ Data preprocessing completed")
        return df
        
    except Exception as e:
        print(f"❌ Error in preprocessing: {str(e)}")
        return df

def get_timeframe_data(df, timeframe, periods_back=0):
    """
    Filter data based on selected timeframe
    timeframe options: 'daily', 'weekly', 'monthly', 'quarterly', 'yearly', '15days'
    """
    if df.empty:
        return df
    
    today = df['Date'].max()
    
    if timeframe == 'daily':
        start_date = today - timedelta(days=periods_back)
        filtered_df = df[df['Date'] == start_date]
    elif timeframe == '15days':
        start_date = today - timedelta(days=15 + periods_back*15)
        end_date = today - timedelta(days=periods_back*15)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    elif timeframe == 'weekly':
        start_date = today - timedelta(weeks=1 + periods_back)
        end_date = today - timedelta(weeks=periods_back)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    elif timeframe == 'monthly':
        start_date = today - pd.DateOffset(months=1 + periods_back)
        end_date = today - pd.DateOffset(months=periods_back)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    elif timeframe == 'quarterly':
        start_date = today - pd.DateOffset(months=3 + periods_back*3)
        end_date = today - pd.DateOffset(months=periods_back*3)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    elif timeframe == 'yearly':
        start_date = today - pd.DateOffset(years=1 + periods_back)
        end_date = today - pd.DateOffset(years=periods_back)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    else:
        filtered_df = df
    
    return filtered_df

# Preprocess the loaded data
if not df.empty:
    df = preprocess_data(df)
    print(f"✅ Preprocessed data ready with {len(df)} records")
    print(f"📈 Available stocks: {sorted(df['Stock Symbol'].unique())}")
else:
    print("❌ No data to preprocess")

## 🚨 1. Broker Dominance Detection

**What it detects:** Single brokers controlling large portions of a stock's trading volume
**Why it matters:** May indicate market manipulation or artificial price movements
**Threshold:** Flags brokers with >60-80% of total volume

In [ ]:
def analyze_broker_dominance(df, stock_symbol=None, timeframe='daily', threshold=60):
    """
    Analyze broker dominance for a specific stock and timeframe
    """
    if df.empty:
        return pd.DataFrame(), {}
    
    # Filter data
    if stock_symbol:
        stock_data = df[df['Stock Symbol'] == stock_symbol]
    else:
        stock_data = df
    
    stock_data = get_timeframe_data(stock_data, timeframe)
    
    if stock_data.empty:
        return pd.DataFrame(), {}
    
    results = []
    
    # Group by stock symbol if analyzing multiple stocks
    for symbol in stock_data['Stock Symbol'].unique():
        symbol_data = stock_data[stock_data['Stock Symbol'] == symbol]
        
        # Calculate total volume for the stock
        total_volume = symbol_data['Quantity'].sum()
        
        # Calculate volume per buyer broker
        buyer_volume = symbol_data.groupby('Buyer')['Quantity'].agg(['sum', 'count']).reset_index()
        buyer_volume.columns = ['Broker', 'Volume', 'Trades']
        buyer_volume['Percentage'] = (buyer_volume['Volume'] / total_volume) * 100
        buyer_volume['Side'] = 'Buyer'
        buyer_volume['Stock'] = symbol
        
        # Calculate volume per seller broker
        seller_volume = symbol_data.groupby('Seller')['Quantity'].agg(['sum', 'count']).reset_index()
        seller_volume.columns = ['Broker', 'Volume', 'Trades']
        seller_volume['Percentage'] = (seller_volume['Volume'] / total_volume) * 100
        seller_volume['Side'] = 'Seller'
        seller_volume['Stock'] = symbol
        
        # Combine buyer and seller data
        all_brokers = pd.concat([buyer_volume, seller_volume], ignore_index=True)
        
        # Flag suspicious brokers
        all_brokers['Suspicious'] = all_brokers['Percentage'] >= threshold
        all_brokers['Total_Stock_Volume'] = total_volume
        
        results.append(all_brokers)
    
    if results:
        final_results = pd.concat(results, ignore_index=True)
        
        # Summary statistics
        summary = {
            'total_records': len(final_results),
            'suspicious_brokers': len(final_results[final_results['Suspicious']]),
            'max_dominance': final_results['Percentage'].max(),
            'avg_dominance': final_results['Percentage'].mean(),
            'stocks_analyzed': final_results['Stock'].nunique()
        }
        
        return final_results.sort_values('Percentage', ascending=False), summary
    
    return pd.DataFrame(), {}

def display_dominance_analysis(df, stock_symbol=None, timeframe='daily'):
    """
    Display broker dominance analysis with visualizations
    """
    results, summary = analyze_broker_dominance(df, stock_symbol, timeframe)
    
    if results.empty:
        print("❌ No data available for analysis")
        return
    
    print(f"📊 BROKER DOMINANCE ANALYSIS - {timeframe.upper()}")
    print("="*50)
    print(f"📈 Stocks Analyzed: {summary['stocks_analyzed']}")
    print(f"🚨 Suspicious Brokers Found: {summary['suspicious_brokers']}")
    print(f"📊 Maximum Dominance: {summary['max_dominance']:.2f}%")
    print(f"📊 Average Dominance: {summary['avg_dominance']:.2f}%")
    
    # Show suspicious brokers
    suspicious = results[results['Suspicious']]
    if not suspicious.empty:
        print("\n🚨 SUSPICIOUS BROKERS (>60% dominance):")
        display(suspicious[['Stock', 'Broker', 'Side', 'Volume', 'Percentage', 'Trades']].round(2))
        
        # Create visualization
        fig = px.bar(suspicious.head(10), 
                    x='Broker', y='Percentage', 
                    color='Side', facet_col='Stock',
                    title='Top 10 Dominant Brokers by Stock',
                    labels={'Percentage': 'Volume Dominance (%)'})
        fig.show()
    else:
        print("\n✅ No suspicious broker dominance detected")
    
    # Show top brokers overall
    print(f"\n📊 TOP 10 BROKERS BY VOLUME:")
    top_brokers = results.head(10)[['Stock', 'Broker', 'Side', 'Volume', 'Percentage', 'Trades']].round(2)
    display(top_brokers)

# Example usage (uncomment to run with your data)
# if not df.empty:
#     display_dominance_analysis(df, timeframe='daily')

## 🔄 2. Wash Trading Detection

**What it detects:** Same brokers appearing on both buy and sell sides, or broker pairs trading repeatedly
**Why it matters:** May indicate illegal wash trading or market manipulation
**Red flags:** Same broker IDs, repetitive broker pairs, circular trading patterns

In [ ]:
def detect_wash_trading(df, stock_symbol=None, timeframe='daily'):
    """
    Detect potential wash trading patterns
    """
    if df.empty:
        return pd.DataFrame(), pd.DataFrame(), {}
    
    # Filter data
    if stock_symbol:
        stock_data = df[df['Stock Symbol'] == stock_symbol]
    else:
        stock_data = df
    
    stock_data = get_timeframe_data(stock_data, timeframe)
    
    if stock_data.empty:
        return pd.DataFrame(), pd.DataFrame(), {}
    
    # 1. Same broker on both sides (exact same broker ID)
    same_broker_trades = stock_data[stock_data['Buyer'] == stock_data['Seller']]
    
    # 2. Broker pairs trading repeatedly
    stock_data['Broker_Pair'] = stock_data[['Buyer', 'Seller']].apply(
        lambda x: '-'.join(sorted([str(x['Buyer']), str(x['Seller'])])), axis=1
    )
    
    # Count trades per broker pair
    pair_counts = stock_data.groupby(['Stock Symbol', 'Broker_Pair']).agg({
        'Quantity': ['sum', 'count'],
        'Amount (Rs)': 'sum',
        'Rate (Rs)': 'mean'
    }).reset_index()
    
    pair_counts.columns = ['Stock', 'Broker_Pair', 'Total_Volume', 'Trade_Count', 'Total_Amount', 'Avg_Rate']
    
    # Flag suspicious pairs (more than 5 trades or high volume)
    suspicious_pairs = pair_counts[
        (pair_counts['Trade_Count'] >= 5) | 
        (pair_counts['Total_Volume'] >= stock_data['Quantity'].quantile(0.9))
    ].sort_values('Trade_Count', ascending=False)
    
    # Summary statistics
    summary = {
        'same_broker_trades': len(same_broker_trades),
        'suspicious_pairs': len(suspicious_pairs),
        'total_broker_pairs': len(pair_counts),
        'max_pair_trades': pair_counts['Trade_Count'].max() if not pair_counts.empty else 0
    }
    
    return same_broker_trades, suspicious_pairs, summary

def display_wash_trading_analysis(df, stock_symbol=None, timeframe='daily'):
    """
    Display wash trading analysis with visualizations
    """
    same_broker, suspicious_pairs, summary = detect_wash_trading(df, stock_symbol, timeframe)
    
    print(f"🔄 WASH TRADING ANALYSIS - {timeframe.upper()}")
    print("="*50)
    print(f"🚨 Same Broker Trades: {summary['same_broker_trades']}")
    print(f"🔄 Suspicious Broker Pairs: {summary['suspicious_pairs']}")
    print(f"📊 Total Broker Pairs: {summary['total_broker_pairs']}")
    print(f"📈 Max Trades per Pair: {summary['max_pair_trades']}")
    
    # Show same broker trades
    if not same_broker.empty:
        print("\n🚨 SAME BROKER ON BOTH SIDES:")
        display(same_broker[['Date', 'Stock Symbol', 'Buyer', 'Seller', 'Quantity', 'Rate (Rs)', 'Amount (Rs)']].head(10))
        
        # Visualization for same broker trades
        same_broker_summary = same_broker.groupby('Stock Symbol')['Quantity'].sum().reset_index()
        if not same_broker_summary.empty:
            fig = px.bar(same_broker_summary, 
                        x='Stock Symbol', y='Quantity',
                        title='Volume of Same-Broker Trades by Stock')
            fig.show()
    else:
        print("\n✅ No same-broker trades detected")
    
    # Show suspicious pairs
    if not suspicious_pairs.empty:
        print(f"\n🔄 SUSPICIOUS BROKER PAIRS:")
        display(suspicious_pairs.head(10))
        
        # Visualization for broker pairs
        fig = px.scatter(suspicious_pairs.head(20), 
                        x='Trade_Count', y='Total_Volume', 
                        color='Stock', size='Total_Amount',
                        title='Suspicious Broker Pairs - Trade Frequency vs Volume',
                        hover_data=['Broker_Pair'])
        fig.show()
    else:
        print("\n✅ No suspicious broker pairs detected")

# Example usage (uncomment to run with your data)
# if not df.empty:
#     display_wash_trading_analysis(df, timeframe='daily')

## 📈 3. Volume Surge & Broker Activity Analysis

**What it detects:** Unusual volume spikes and which brokers are responsible
**Why it matters:** Sudden volume surges often precede price manipulation
**Analysis:** Compares current volume vs 10-day average, identifies new active brokers

In [ ]:
def analyze_volume_surge(df, stock_symbol=None, surge_threshold=2.0):
    """
    Analyze volume surges and responsible brokers
    """
    if df.empty:
        return pd.DataFrame(), {}
    
    # Filter for specific stock if provided
    if stock_symbol:
        stock_data = df[df['Stock Symbol'] == stock_symbol]
    else:
        stock_data = df
    
    results = []
    
    for symbol in stock_data['Stock Symbol'].unique():
        symbol_data = stock_data[stock_data['Stock Symbol'] == symbol].copy()
        symbol_data = symbol_data.sort_values('Date')
        
        # Calculate daily volume
        daily_volume = symbol_data.groupby('Date')['Quantity'].sum().reset_index()
        daily_volume['Stock'] = symbol
        
        # Calculate 10-day rolling average
        daily_volume['Volume_10day_avg'] = daily_volume['Quantity'].rolling(window=10, min_periods=1).mean()
        daily_volume['Volume_Ratio'] = daily_volume['Quantity'] / daily_volume['Volume_10day_avg']
        
        # Identify surge days
        surge_days = daily_volume[daily_volume['Volume_Ratio'] >= surge_threshold]
        
        # For each surge day, identify responsible brokers
        for _, surge_day in surge_days.iterrows():
            surge_date = surge_day['Date']
            surge_trades = symbol_data[symbol_data['Date'] == surge_date]
            
            # Analyze broker activity on surge day
            buyer_activity = surge_trades.groupby('Buyer')['Quantity'].sum().reset_index()
            buyer_activity.columns = ['Broker', 'Volume']
            buyer_activity['Side'] = 'Buyer'
            buyer_activity['Date'] = surge_date
            buyer_activity['Stock'] = symbol
            buyer_activity['Volume_Ratio'] = surge_day['Volume_Ratio']
            buyer_activity['Total_Day_Volume'] = surge_day['Quantity']
            buyer_activity['Broker_Percentage'] = (buyer_activity['Volume'] / surge_day['Quantity']) * 100
            
            seller_activity = surge_trades.groupby('Seller')['Quantity'].sum().reset_index()
            seller_activity.columns = ['Broker', 'Volume']
            seller_activity['Side'] = 'Seller'
            seller_activity['Date'] = surge_date
            seller_activity['Stock'] = symbol
            seller_activity['Volume_Ratio'] = surge_day['Volume_Ratio']
            seller_activity['Total_Day_Volume'] = surge_day['Quantity']
            seller_activity['Broker_Percentage'] = (seller_activity['Volume'] / surge_day['Quantity']) * 100
            
            # Check for new brokers (brokers not active in previous 10 days)
            prev_10_days = symbol_data[
                (symbol_data['Date'] >= surge_date - timedelta(days=10)) & 
                (symbol_data['Date'] < surge_date)
            ]
            
            prev_buyers = set(prev_10_days['Buyer'].unique())
            prev_sellers = set(prev_10_days['Seller'].unique())
            
            buyer_activity['New_Broker'] = ~buyer_activity['Broker'].isin(prev_buyers)
            seller_activity['New_Broker'] = ~seller_activity['Broker'].isin(prev_sellers)
            
            results.extend([buyer_activity, seller_activity])
    
    if results:
        final_results = pd.concat(results, ignore_index=True)
        
        summary = {
            'surge_days': len(final_results['Date'].unique()),
            'avg_surge_ratio': final_results['Volume_Ratio'].mean(),
            'max_surge_ratio': final_results['Volume_Ratio'].max(),
            'new_brokers_count': len(final_results[final_results['New_Broker']]),
            'stocks_with_surges': final_results['Stock'].nunique()
        }
        
        return final_results, summary
    
    return pd.DataFrame(), {}

def display_volume_surge_analysis(df, stock_symbol=None):
    """
    Display volume surge analysis with visualizations
    """
    results, summary = analyze_volume_surge(df, stock_symbol)
    
    if results.empty:
        print("❌ No volume surge data available")
        return
    
    print(f"📈 VOLUME SURGE ANALYSIS")
    print("="*50)
    print(f"📊 Surge Days Detected: {summary['surge_days']}")
    print(f"📈 Average Surge Ratio: {summary['avg_surge_ratio']:.2f}x")
    print(f"🚀 Maximum Surge Ratio: {summary['max_surge_ratio']:.2f}x")
    print(f"🆕 New Brokers in Surges: {summary['new_brokers_count']}")
    print(f"📊 Stocks with Surges: {summary['stocks_with_surges']}")
    
    # Show biggest surges
    biggest_surges = results.sort_values('Volume_Ratio', ascending=False).drop_duplicates(['Date', 'Stock']).head(10)
    print(f"\n🚀 BIGGEST VOLUME SURGES:")
    display(biggest_surges[['Date', 'Stock', 'Volume_Ratio', 'Total_Day_Volume']].round(2))
    
    # Show new brokers
    new_brokers = results[results['New_Broker']].sort_values('Volume', ascending=False)
    if not new_brokers.empty:
        print(f"\n🆕 NEW BROKERS IN SURGES:")
        display(new_brokers[['Date', 'Stock', 'Broker', 'Side', 'Volume', 'Broker_Percentage']].head(10).round(2))
        
        # Visualization
        fig = px.scatter(new_brokers.head(20), 
                        x='Date', y='Volume', 
                        color='Stock', size='Broker_Percentage',
                        title='New Broker Activity During Volume Surges',
                        hover_data=['Broker', 'Side'])
        fig.show()
    else:
        print("\n✅ No new brokers detected in volume surges")
    
    # Show top contributing brokers
    top_contributors = results.groupby(['Broker', 'Side'])['Volume'].sum().reset_index().sort_values('Volume', ascending=False)
    print(f"\n📊 TOP BROKERS IN VOLUME SURGES:")
    display(top_contributors.head(10))

# Example usage (uncomment to run with your data)
# if not df.empty:
#     display_volume_surge_analysis(df)

## 💰 4. Aggressive Buying Price Analysis

**What it detects:** Brokers consistently buying at higher prices to drive market up
**Why it matters:** May indicate artificial price pumping or manipulation
**Analysis:** Compares broker buy prices vs average market price

In [ ]:
def analyze_aggressive_buying(df, stock_symbol=None, timeframe='daily', aggressiveness_threshold=5):
    """
    Analyze aggressive buying behavior (buying at consistently higher prices)
    """
    if df.empty:
        return pd.DataFrame(), {}
    
    # Filter data
    if stock_symbol:
        stock_data = df[df['Stock Symbol'] == stock_symbol]
    else:
        stock_data = df
    
    stock_data = get_timeframe_data(stock_data, timeframe)
    
    if stock_data.empty:
        return pd.DataFrame(), {}
    
    results = []
    
    for symbol in stock_data['Stock Symbol'].unique():
        symbol_data = stock_data[stock_data['Stock Symbol'] == symbol].copy()
        
        # Calculate daily average price for the stock
        daily_avg_price = symbol_data.groupby('Date')['Rate (Rs)'].mean().reset_index()
        daily_avg_price.columns = ['Date', 'Market_Avg_Price']
        
        # Merge with original data
        symbol_data = symbol_data.merge(daily_avg_price, on='Date')
        
        # Analyze broker buying behavior
        broker_analysis = symbol_data.groupby(['Buyer', 'Date']).agg({
            'Rate (Rs)': ['mean', 'count'],
            'Quantity': 'sum',
            'Market_Avg_Price': 'first'
        }).reset_index()
        
        broker_analysis.columns = ['Broker', 'Date', 'Avg_Buy_Price', 'Trade_Count', 'Volume', 'Market_Avg_Price']
        broker_analysis['Stock'] = symbol
        broker_analysis['Price_Premium'] = ((broker_analysis['Avg_Buy_Price'] - broker_analysis['Market_Avg_Price']) / broker_analysis['Market_Avg_Price']) * 100
        
        # Flag aggressive buyers (consistently buying above market price)
        broker_summary = broker_analysis.groupby('Broker').agg({
            'Price_Premium': ['mean', 'std', 'count'],
            'Volume': 'sum',
            'Trade_Count': 'sum'
        }).reset_index()
        
        broker_summary.columns = ['Broker', 'Avg_Price_Premium', 'Price_Premium_Std', 'Trading_Days', 'Total_Volume', 'Total_Trades']
        broker_summary['Stock'] = symbol
        broker_summary['Aggressive'] = (broker_summary['Avg_Price_Premium'] >= aggressiveness_threshold) & (broker_summary['Trading_Days'] >= 2)
        
        results.append(broker_summary)
    
    if results:
        final_results = pd.concat(results, ignore_index=True)
        
        summary = {
            'aggressive_brokers': len(final_results[final_results['Aggressive']]),
            'avg_price_premium': final_results['Avg_Price_Premium'].mean(),
            'max_price_premium': final_results['Avg_Price_Premium'].max(),
            'total_brokers_analyzed': len(final_results)
        }
        
        return final_results, summary
    
    return pd.DataFrame(), {}

def display_aggressive_buying_analysis(df, stock_symbol=None, timeframe='daily'):
    """
    Display aggressive buying analysis with visualizations
    """
    results, summary = analyze_aggressive_buying(df, stock_symbol, timeframe)
    
    if results.empty:
        print("❌ No aggressive buying data available")
        return
    
    print(f"💰 AGGRESSIVE BUYING ANALYSIS - {timeframe.upper()}")
    print("="*50)
    print(f"🚨 Aggressive Brokers: {summary['aggressive_brokers']}")
    print(f"📊 Average Price Premium: {summary['avg_price_premium']:.2f}%")
    print(f"📈 Maximum Price Premium: {summary['max_price_premium']:.2f}%")
    print(f"📊 Total Brokers Analyzed: {summary['total_brokers_analyzed']}")
    
    # Show aggressive brokers
    aggressive_brokers = results[results['Aggressive']].sort_values('Avg_Price_Premium', ascending=False)
    if not aggressive_brokers.empty:
        print(f"\n🚨 AGGRESSIVE BUYERS (>5% price premium):")
        display(aggressive_brokers[['Stock', 'Broker', 'Avg_Price_Premium', 'Total_Volume', 'Trading_Days']].round(2))
        
        # Visualization
        fig = px.scatter(aggressive_brokers, 
                        x='Total_Volume', y='Avg_Price_Premium',
                        color='Stock', size='Trading_Days',
                        title='Aggressive Buying Behavior - Volume vs Price Premium',
                        hover_data=['Broker'])
        fig.show()
    else:
        print("\n✅ No aggressive buying behavior detected")
    
    # Show top price premium brokers
    top_premium = results.sort_values('Avg_Price_Premium', ascending=False).head(10)
    print(f"\n📊 TOP PRICE PREMIUM BROKERS:")
    display(top_premium[['Stock', 'Broker', 'Avg_Price_Premium', 'Total_Volume', 'Trading_Days']].round(2))
    
    # Price premium distribution
    fig = px.histogram(results, x='Avg_Price_Premium', 
                      title='Distribution of Broker Price Premiums',
                      labels={'Avg_Price_Premium': 'Average Price Premium (%)'})
    fig.show()

# Example usage (uncomment to run with your data)
# if not df.empty:
#     display_aggressive_buying_analysis(df, timeframe='daily')

## 📋 5. Net Holdings & Share Turnover Analysis

**What it shows:** Broker net positions and overall share trading activity
**Why it matters:** Helps understand broker strategies and market liquidity
**Analysis:** Net buying/selling per broker, total share turnover, market participation

In [ ]:
def analyze_net_holdings(df, stock_symbol=None, timeframe='daily'):
    """
    Analyze net holdings and share turnover
    """
    if df.empty:
        return pd.DataFrame(), pd.DataFrame(), {}
    
    # Filter data
    if stock_symbol:
        stock_data = df[df['Stock Symbol'] == stock_symbol]
    else:
        stock_data = df
    
    stock_data = get_timeframe_data(stock_data, timeframe)
    
    if stock_data.empty:
        return pd.DataFrame(), pd.DataFrame(), {}
    
    # Broker net position analysis
    broker_positions = []
    
    for symbol in stock_data['Stock Symbol'].unique():
        symbol_data = stock_data[stock_data['Stock Symbol'] == symbol]
        
        # Calculate buy positions
        buy_positions = symbol_data.groupby('Buyer').agg({
            'Quantity': 'sum',
            'Amount (Rs)': 'sum',
            'Rate (Rs)': 'mean'
        }).reset_index()
        buy_positions.columns = ['Broker', 'Buy_Volume', 'Buy_Amount', 'Avg_Buy_Price']
        
        # Calculate sell positions
        sell_positions = symbol_data.groupby('Seller').agg({
            'Quantity': 'sum',
            'Amount (Rs)': 'sum',
            'Rate (Rs)': 'mean'
        }).reset_index()
        sell_positions.columns = ['Broker', 'Sell_Volume', 'Sell_Amount', 'Avg_Sell_Price']
        
        # Merge buy and sell positions
        all_brokers = set(buy_positions['Broker'].tolist() + sell_positions['Broker'].tolist())
        
        net_positions = []
        for broker in all_brokers:
            buy_vol = buy_positions[buy_positions['Broker'] == broker]['Buy_Volume'].sum()
            sell_vol = sell_positions[sell_positions['Broker'] == broker]['Sell_Volume'].sum()
            buy_amt = buy_positions[buy_positions['Broker'] == broker]['Buy_Amount'].sum()
            sell_amt = sell_positions[sell_positions['Broker'] == broker]['Sell_Amount'].sum()
            
            avg_buy_price = buy_positions[buy_positions['Broker'] == broker]['Avg_Buy_Price'].mean()
            avg_sell_price = sell_positions[sell_positions['Broker'] == broker]['Avg_Sell_Price'].mean()
            
            net_positions.append({
                'Broker': broker,
                'Stock': symbol,
                'Buy_Volume': buy_vol,
                'Sell_Volume': sell_vol,
                'Net_Volume': buy_vol - sell_vol,
                'Buy_Amount': buy_amt,
                'Sell_Amount': sell_amt,
                'Net_Amount': buy_amt - sell_amt,
                'Avg_Buy_Price': avg_buy_price if not pd.isna(avg_buy_price) else 0,
                'Avg_Sell_Price': avg_sell_price if not pd.isna(avg_sell_price) else 0,
                'Total_Volume': buy_vol + sell_vol,
                'Position_Type': 'Net_Buyer' if buy_vol > sell_vol else 'Net_Seller' if sell_vol > buy_vol else 'Balanced'
            })
        
        broker_positions.extend(net_positions)
    
    # Share turnover analysis
    share_turnover = stock_data.groupby('Stock Symbol').agg({
        'Quantity': 'sum',
        'Amount (Rs)': 'sum',
        'Rate (Rs)': ['mean', 'min', 'max', 'std'],
        'Contract No.': 'count'
    }).reset_index()
    
    share_turnover.columns = ['Stock', 'Total_Volume', 'Total_Amount', 'Avg_Price', 'Min_Price', 'Max_Price', 'Price_Volatility', 'Total_Trades']
    share_turnover['Price_Range'] = share_turnover['Max_Price'] - share_turnover['Min_Price']
    share_turnover['Price_Range_Pct'] = (share_turnover['Price_Range'] / share_turnover['Avg_Price']) * 100
    
    # Summary statistics
    broker_df = pd.DataFrame(broker_positions)
    summary = {
        'total_brokers': len(broker_df['Broker'].unique()) if not broker_df.empty else 0,
        'net_buyers': len(broker_df[broker_df['Position_Type'] == 'Net_Buyer']) if not broker_df.empty else 0,
        'net_sellers': len(broker_df[broker_df['Position_Type'] == 'Net_Seller']) if not broker_df.empty else 0,
        'balanced_brokers': len(broker_df[broker_df['Position_Type'] == 'Balanced']) if not broker_df.empty else 0,
        'total_market_volume': share_turnover['Total_Volume'].sum() if not share_turnover.empty else 0,
        'avg_price_volatility': share_turnover['Price_Range_Pct'].mean() if not share_turnover.empty else 0
    }
    
    return broker_df, share_turnover, summary

def display_net_holdings_analysis(df, stock_symbol=None, timeframe='daily'):
    """
    Display net holdings and turnover analysis
    """
    broker_positions, share_turnover, summary = analyze_net_holdings(df, stock_symbol, timeframe)
    
    if broker_positions.empty:
        print("❌ No holdings data available")
        return
    
    print(f"📋 NET HOLDINGS & TURNOVER ANALYSIS - {timeframe.upper()}")
    print("="*60)
    print(f"👥 Total Brokers: {summary['total_brokers']}")
    print(f"📈 Net Buyers: {summary['net_buyers']}")
    print(f"📉 Net Sellers: {summary['net_sellers']}")
    print(f"⚖️ Balanced Brokers: {summary['balanced_brokers']}")
    print(f"📊 Total Market Volume: {summary['total_market_volume']:,.0f}")
    print(f"📊 Average Price Volatility: {summary['avg_price_volatility']:.2f}%")
    
    # Top net buyers
    top_buyers = broker_positions[broker_positions['Position_Type'] == 'Net_Buyer'].sort_values('Net_Volume', ascending=False)
    if not top_buyers.empty:
        print(f"\n📈 TOP NET BUYERS:")
        display(top_buyers[['Stock', 'Broker', 'Net_Volume', 'Net_Amount', 'Total_Volume']].head(10))
        
        # Visualization for net buyers
        fig = px.bar(top_buyers.head(15), 
                    x='Broker', y='Net_Volume', 
                    color='Stock',
                    title='Top Net Buyers by Volume')
        fig.update_xaxes(tickangle=45)
        fig.show()
    
    # Top net sellers
    top_sellers = broker_positions[broker_positions['Position_Type'] == 'Net_Seller'].sort_values('Net_Volume')
    if not top_sellers.empty:
        print(f"\n📉 TOP NET SELLERS:")
        display(top_sellers[['Stock', 'Broker', 'Net_Volume', 'Net_Amount', 'Total_Volume']].head(10))
    
    # Share turnover analysis
    if not share_turnover.empty:
        print(f"\n📊 SHARE TURNOVER ANALYSIS:")
        share_turnover_display = share_turnover.sort_values('Total_Volume', ascending=False)
        display(share_turnover_display[['Stock', 'Total_Volume', 'Total_Amount', 'Avg_Price', 'Price_Range_Pct', 'Total_Trades']].round(2))
        
        # Visualization for share turnover
        fig = make_subplots(rows=2, cols=1, 
                           subplot_titles=['Volume by Stock', 'Price Volatility by Stock'])
        
        fig.add_trace(go.Bar(x=share_turnover_display['Stock'], y=share_turnover_display['Total_Volume'], 
                            name='Volume'), row=1, col=1)
        fig.add_trace(go.Bar(x=share_turnover_display['Stock'], y=share_turnover_display['Price_Range_Pct'], 
                            name='Price Volatility %'), row=2, col=1)
        
        fig.update_layout(title='Share Trading Summary', showlegend=False)
        fig.show()
    
    # Broker activity distribution
    activity_dist = broker_positions['Position_Type'].value_counts()
    fig = px.pie(values=activity_dist.values, names=activity_dist.index, 
                title='Broker Position Distribution')
    fig.show()

# Example usage (uncomment to run with your data)
# if not df.empty:
#     display_net_holdings_analysis(df, timeframe='daily')

## 🎛️ Interactive Analysis Dashboard

**Easy-to-use controls for non-technical users**
Select your preferred stock and timeframe to run all analyses automatically.

In [ ]:
def create_interactive_dashboard(df):
    """
    Create an interactive dashboard for easy analysis
    """
    if df.empty:
        print("❌ No data available for dashboard")
        return
    
    # Get available stocks
    available_stocks = ['All Stocks'] + sorted(df['Stock Symbol'].unique().tolist())
    
    # Create widgets
    stock_dropdown = widgets.Dropdown(
        options=available_stocks,
        value='All Stocks',
        description='Stock:',
        style={'description_width': 'initial'}
    )
    
    timeframe_dropdown = widgets.Dropdown(
        options=['daily', 'weekly', '15days', 'monthly', 'quarterly', 'yearly'],
        value='daily',
        description='Timeframe:',
        style={'description_width': 'initial'}
    )
    
    analysis_type = widgets.Dropdown(
        options=[
            'All Analyses',
            'Broker Dominance',
            'Wash Trading',
            'Volume Surge',
            'Aggressive Buying',
            'Net Holdings'
        ],
        value='All Analyses',
        description='Analysis:',
        style={'description_width': 'initial'}
    )
    
    run_button = widgets.Button(
        description='🔍 Run Analysis',
        button_style='success',
        style={'font_weight': 'bold'}
    )
    
    output = widgets.Output()
    
    def run_analysis(button):
        with output:
            clear_output()
            
            selected_stock = None if stock_dropdown.value == 'All Stocks' else stock_dropdown.value
            selected_timeframe = timeframe_dropdown.value
            selected_analysis = analysis_type.value
            
            print(f"🔍 Running {selected_analysis} for {stock_dropdown.value} - {selected_timeframe}")
            print("="*70)
            
            try:
                if selected_analysis == 'All Analyses' or selected_analysis == 'Broker Dominance':
                    display_dominance_analysis(df, selected_stock, selected_timeframe)
                    print("\\n" + "="*70 + "\\n")
                
                if selected_analysis == 'All Analyses' or selected_analysis == 'Wash Trading':
                    display_wash_trading_analysis(df, selected_stock, selected_timeframe)
                    print("\\n" + "="*70 + "\\n")
                
                if selected_analysis == 'All Analyses' or selected_analysis == 'Volume Surge':
                    display_volume_surge_analysis(df, selected_stock)
                    print("\\n" + "="*70 + "\\n")
                
                if selected_analysis == 'All Analyses' or selected_analysis == 'Aggressive Buying':
                    display_aggressive_buying_analysis(df, selected_stock, selected_timeframe)
                    print("\\n" + "="*70 + "\\n")
                
                if selected_analysis == 'All Analyses' or selected_analysis == 'Net Holdings':
                    display_net_holdings_analysis(df, selected_stock, selected_timeframe)
                
                print("\\n✅ Analysis completed!")
                
            except Exception as e:
                print(f"❌ Error during analysis: {str(e)}")
    
    run_button.on_click(run_analysis)
    
    # Create layout
    controls = widgets.HBox([stock_dropdown, timeframe_dropdown, analysis_type, run_button])
    dashboard = widgets.VBox([
        widgets.HTML("<h3>🎛️ NEPSE Floor Sheet Analysis Dashboard</h3>"),
        controls,
        output
    ])
    
    return dashboard

# Create and display the dashboard
if not df.empty:
    dashboard = create_interactive_dashboard(df)
    display(dashboard)
    
    print("\\n" + "="*70)
    print("🎯 QUICK START GUIDE:")
    print("1. Select a stock from the dropdown (or 'All Stocks' for market-wide analysis)")
    print("2. Choose your preferred timeframe")
    print("3. Pick a specific analysis or run 'All Analyses'")
    print("4. Click 'Run Analysis' to see results")
    print("="*70)
else:
    print("❌ No data available. Please load floor sheet data first.")

## 📊 Summary & Key Insights

### 🎯 What This Analysis Tells You:

1. **🚨 Broker Dominance** - Identifies potential market manipulation when single brokers control large volumes
2. **🔄 Wash Trading** - Detects suspicious same-broker trades that may indicate illegal activity  
3. **📈 Volume Surges** - Highlights unusual trading spikes and responsible brokers
4. **💰 Aggressive Buying** - Finds brokers consistently buying at premium prices
5. **📋 Net Holdings** - Shows overall broker positions and market participation

### 🔍 How to Interpret Results:

- **Red Flags:** High broker dominance (>60%), same-broker trades, sudden volume spikes
- **Green Flags:** Balanced broker participation, normal volume patterns, fair pricing
- **Watch List:** New brokers with high activity, repetitive trading patterns

### ⚠️ Important Notes:

- This analysis is for educational and research purposes
- Always consider market context and fundamentals
- Suspicious patterns don't always indicate wrongdoing
- Consult financial professionals for investment decisions

---

**🎉 Analysis Complete! Use the interactive dashboard above to explore different stocks and timeframes.**